# Build a Recommendation System for Purchase Data

## Settings

Prepare the enviroments with libraries and utility functions

In [ ]:
#Libraries

#Data
import sqlalchemy as sql

#Data Science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor, BaselineOnly
from surprise import KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline
from surprise import SVD,SVDpp,NMF
from surprise import SlopeOne, CoClustering
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.accuracy import rmse, mae
from surprise import dump

#Model Tracking
import mlflow
from mlflow.tracking import MlflowClient

#Utils
import os
import glob
import configparser
from collections import defaultdict
import tempfile
import time
import json
import base64
from io import BytesIO

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Enviroment Variables
dbconnPath = './dbconn.properties'
interdata = '../data/interim/'
outdata = '../data/processed/'
outmodels = '../models'
outreports = '../reports/'

# Set dbconnection variables
config = configparser.RawConfigParser()
config.read(dbconnPath)
params = config
db_host=params.get('CONN', 'host')
db_port=params.get('CONN', 'port')
db_user=params.get('CONN', 'user')
db_pwd=params.get('CONN', 'password')
db_name=params.get('CONN', 'database')

# Set connection string
connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'

## Data Preparation

We have to:

1) Split each list of items in the products column into rows

2) Count the number of products bought by a user

In [ ]:
# connect to database
engine = sql.create_engine(connection_str)
connection = engine.connect()

In [ ]:
print(f'{db_name} contains {engine.table_names()[0]}, {engine.table_names()[1]} tables')
# print(f'{db_name} contains {engine.table_names()[0]} table')

In [ ]:
# Convert tables in dataframes (Don't do that if you have a large dataset. Use just samples)
# df_cus = pd.read_sql("select * from CUSTOMERID", connection)
df_trx = pd.read_sql("select * from TRANSACTIONS", connection)
connection.close()

In [ ]:
# print(df_cus.shape)
# df_cus.head()

In [ ]:
print(df_trx.shape)
df_trx.head()

In [ ]:
data_prep_1=pd.DataFrame(df_trx.products.str.split('|').tolist(), index=df_trx.customerId)\
.stack()\
.reset_index()\
.groupby(['customerId', 0])\
.agg({0: 'count'})\
.rename(columns={0: 'purchase_count'})\
.reset_index()\
.rename(columns={0: 'productId'})

# Store 
data_prep_1.to_csv("".join([interdata, 'syntetic_purchase_count.csv']))

In [ ]:
print(data_prep_1.head())
print('*'*40)
print(data_prep_1.info())

In [ ]:
#Let's normalize data in a way to have a rank
data_prep_2 = pd.pivot_table(data=data_prep_1, index='customerId', columns='productId', values='purchase_count', aggfunc='sum')
data_prep_3 = (data_prep_2 - data_prep_2.min())/(data_prep_2.max() - data_prep_2.min())
data_prep_4 = data_prep_3.reset_index().melt(id_vars=['customerId'], value_name='prod_ratings').dropna()
data_prep_4.index = np.arange(0, len(data_prep_4))
data_prep_4['prod_ratings'] = data_prep_4['prod_ratings'].apply(lambda x: int((round(x, 2))*10))

#Store
data_prep_4.to_csv("".join([interdata, 'syntetic_prod_ratings.csv']))

In [ ]:
print(data_prep_4.sort_values(by=['customerId']).head())

Then, we get **Triplet Representation** with each column representing user, item and the given rating respectively.
Notice the rating goes from 0–100 (with 100 being the most number of purchase for an item and 0 being 0 purchase count for that item).
It's a kind of preference for each user


## Data Exploration

Explore syntetic ratings by UserID and ProductID

### Rating Distributions

In [ ]:
df_plt1=data_prep_4['prod_ratings']
fig, ax = plt.subplots(figsize=(7, 3))
plt.hist(df_plt1)
ax.set_xlabel('Syntetic Product Ratings')
ax.set_ylabel('Frequency')
ax.set_title('Rating Distribution of {} items'.format(data_prep_4.shape[0]))
plt.show()

### Rating Distribution by ProductId

In [ ]:
df_plt_2=data_prep_4.groupby('productId')['prod_ratings'].count().reset_index()[0:50]
df_plt_2
fig, ax = plt.subplots(figsize=(7, 3))
plt.bar(df_plt_2.productId, df_plt_2.prod_ratings, )
ax.set_xlabel('Number of Ratings per product')
ax.set_ylabel('Count')
ax.set_title('Rating Distribution by ProductId')
plt.show()

In [ ]:
df_plt_2.sort_values(by='prod_ratings', ascending=False)[0:10]

In [ ]:
df_plt_2=data_prep_4.groupby('productId')['prod_ratings'].count()
df_plt_2
fig, ax = plt.subplots(figsize=(7, 3))
plt.hist(df_plt_2)
ax.set_xlim(0, 3000)
ax.set_xlabel('Number of Ratings')
ax.set_ylabel('Count')
ax.set_title('Rating Distribution by ProductId')
plt.show()

### Ratings Distribution By User

In [ ]:
df_plt_3=data_prep_4.groupby('customerId')['prod_ratings'].count().reset_index()[0:50]
fig, ax = plt.subplots(figsize=(7, 3))
plt.bar(df_plt_3.customerId, df_plt_3.prod_ratings)
ax.set_xlabel('Number of Ratings per Customer')
ax.set_ylabel('Count')
ax.set_title('Rating Distribution by customerId')
plt.show()

In [ ]:
df_plt_3.sort_values(by='prod_ratings', ascending=False)[0:10]

In [ ]:
df_plt_3=data_prep_4.groupby('customerId')['prod_ratings'].count()
fig, ax = plt.subplots(figsize=(7, 3))
plt.hist(df_plt_3)
ax.set_xlim(0, 50)
ax.set_xlabel('Number of Ratings per Customer')
ax.set_ylabel('Frequency')
ax.set_title('Rating Distribution by customerId')
plt.show()

## Model Training and Evaluation

To do that, I use **Surprise** which is an easy-to-use Python scikit for recommender systems.

I follow the Getting Started (https://surprise.readthedocs.io/en/stable/getting_started.html#getting-started)

Reference: https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

### Settings and Utility Functions (mlflow_tracker, get_Iu, get_Ui)

In [ ]:
# The columns must correspond to user id, item id and ratings (in that order)
data_prep_5 = data_prep_4.rename(columns={'customerId': 'userID', 'productId':'itemID', 'prod_ratings':'rating'})
data_prep_5.to_csv(os.path.join(outdata, 'abt.csv'))

#Create the experiment
mlflow.set_experiment('Purchase Recommended System')

#Create Minio Bucket for Artefact Storage
%run -t 00_set_bucket.ipynb


def mlflow_tracker(data, algo, params, rundesc='myrun'):
    """ 
    return the run id and experiment id of each model
    args: 
      reader_data: data parsed with Reader class 
      params: paramters of cross_validate function
      rundesc: A description to populate exoeriment note
      
    returns: 
      run_id and experiment_id
    """
        
    # Store Algo name
    algo_name = 'CrossValidation' + str(algo.__class__.__name__)

    with mlflow.start_run(run_name=algo_name) as run:

        # Store run_id and experiment_id
        run_id=run.info.run_uuid
        experiment_id=run.info.experiment_id
        
        #Read data
        mindata = data.rating.min()
        maxdata = data.rating.max()
        reader = Reader(rating_scale=(mindata,maxdata))
        data_parse = Dataset.load_from_df(data[['userID', 'itemID', 'rating']], reader)
        
        #Create model instance on a 3-fold cross validation
        redic=cross_validate(algo, data_parse, **params)
        #Create a dataframe of means
        recdf=pd.DataFrame.from_dict(redic).mean(axis=0)

        #Log params
        mlflow.log_params(params)
        mlflow.log_metric('test_time', recdf.iloc[3])
        mlflow.log_metric('test_rmse_mean', recdf.iloc[0])
        mlflow.log_metric('test_mae_mean', recdf.iloc[1])
        mlflow.log_metric('fit_time', recdf.iloc[2])     
        
        # Set the notes for Runs
        MlflowClient().set_tag(run_id,
                               "mlflow.note.content",
                               rundesc)

    return (run_id, experiment_id)

### Basic usage: Automatic cross-validation

#### Basic Algorithms

##### random_pred.NormalPredictor
Algorithm predicting a random rating based on the distribution of the training set, which is assumed to be normal.
##### baseline_only.BaselineOnly
Algorithm predicting the baseline estimate for given user and item.

#### KNN algorithms

##### knns.KNNBasic
A basic collaborative filtering algorithm
##### knns.KNNWithMeans
A basic collaborative filtering algorithm, taking into account the mean ratings of each user.
##### knns.KNNWithZScore
A basic collaborative filtering algorithm, taking into account taking into account the z-score normalization of each user.
##### knns.KNNBaseline
A basic collaborative filtering algorithm taking into account a baseline rating.

#### Matrix Factorization-based algorithms
##### matrix_factorization.SVD
The famous SVD algorithm, as popularized by Simon Funk during the Netflix Prize.When baselines are not used, this is equivalent to Probabilistic Matrix Factorization
##### matrix_factorization.SVDpp
The SVD++ algorithm, an extension of SVD taking into account implicit ratings.
##### matrix_factorization.NMF
A collaborative filtering algorithm based on Non-negative Matrix Factorization.

#### Other Algorithms
##### slope_one.SlopeOne
A simple yet accurate collaborative filtering algorithm.
##### co_clustering.CoClustering
A collaborative filtering algorithm based on co-clustering.



In [ ]:
params = {'measures': ['RMSE', 'MAE'], 'cv':3, 'verbose': False}
algos = [BaselineOnly()]

# , NormalPredictor(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline(), SVD(), SVDpp(), SlopeOne(), CoClustering()

for algo in algos:
    mlflow_tracker(data_prep_5, algo, params)
    time.sleep(1)

**Comment:  Based on the Mlflow tracking service, the best model is BaselineOnly algorithm**

### Tuning the Best Model

In [ ]:
def reader(df):
    """
    return a data parsed with Reader class
    args:
        df: pandas dataframe
    returns:
        data parsed
    """
    mindata = df.rating.min()
    maxdata = df.rating.max()
    reader = Reader(rating_scale=(mindata,maxdata))
    data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
    return data
    
def preparer(data, method=None, test_size=0.25):
    """ 
    return train and test sets 
    args: 
      data: data parsed with Reader class
      method: for sampling data. 'train_test_split' option. Default None
    returns: 
      trainset and testset
    """
    if method == 'train_test_split':
        trainset, testset = train_test_split(data, test_size=test_size)
    trainset = data.build_full_trainset()
    testset = trainset.build_testset()
    
    return trainset, testset

def trainer(trainset, bsl_options):
    """ 
    return trained model 
    args: 
      trainset: training data parsed with Reader class
      bsl_option: algorithm options 
    returns: 
      trained model
    """
    algo = BaselineOnly(bsl_options=bsl_options)
    model = algo.fit(trainset)
    return model

def predictor(model, testset):
    """ 
    return trained model and predictions
    args: 
      model: trained model
      testset: test data parsed with Reader class 
    returns: 
      trained model and predictions
    """
    predictions = model.test(testset)
    return model, predictions

def tuner(data, bsl_options_grid, param_model):
    """ 
    return model parameters, tuning history and best tuned model
    args: 
      data: data parsed with Reader class 
      bsl_options_grid:  algorithm options grid
      param_model: error measures and cross validation parameters 
    returns: 
      param_model, history_tune, tuned_model
    """
    
    gs = GridSearchCV(BaselineOnly, bsl_options_grid, **param_model)
    gs.fit(data)
    
    history_tune=pd.DataFrame.from_dict(gs.cv_results)
    
    best_bsl_options = gs.best_params['rmse']
    
    return param_model, history_tune

 # Define utils functions for prediction readability

def get_Iu(trainset, uid):
    """
    return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0

def get_Ui(trainset, iid):
    """
    return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
def plotter_hist(data, productId):
    plt.hist(data_prep_4.loc[data_prep_4['productId'] == '2']['prod_ratings'])
    plt.xlabel('rating')
    plt.ylabel('Number of ratings')
    plt.title('Number of ratings 2 has received')

def mlflow_tune_tracker(data, algo_name, param_grid, param_model, method=None, rundesc='myruntuned'):
    """ 
    return the run id and experiment id of tuned model
    args: 
      algo_name: the name of tuned algorithm 
      param_grid:  algorithm options grid
      param_model: error measures and cross validation parameters 
    returns: 
      run_id, experiment_id
    """
    
    with mlflow.start_run(run_name=algo_name) as run:

        # Store run_id and experiment_id
        run_id=run.info.run_uuid
        experiment_id=run.info.experiment_id
        
        #Read data
        data_parse = reader(data)

        #Tune
        params, history_tune = tuner(data_parse, param_grid, param_model)

        #History
        for index, row in history_tune.iterrows():
            with mlflow.start_run(experiment_id=experiment_id, run_name=algo_name + str(index), nested=True) as subruns:

                #Set variables 
                bsl_options = row['params']
                params_tune = {**params, **bsl_options}
                trainset, testset = preparer(data_parse, method)

                #Log params
                mlflow.log_params(params_tune)
                mlflow.log_metric('fit_time',round(row['mean_fit_time'], 3))
                mlflow.log_metric('test_time', round(row['mean_test_time'], 3))
                mlflow.log_metric('test_rmse_mean', round(row['mean_test_rmse'], 3))
                mlflow.log_metric('test_mae_mean', round(row['mean_test_mae'], 3))
                
                #Log Model (artefact)
                temp = tempfile.NamedTemporaryFile(prefix="model_", suffix=".pkl")
                temp_name = temp.name
                try:
                    model, predictions = predictor(trainer(trainset, bsl_options), testset)
                    dump.dump(temp_name, predictions, model)
                    mlflow.log_artifact(temp_name, 'model')
                finally:
                    temp.close()

                 #Log best and worst predictions. Log charts for validation
                df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
                df['Iu'] = [get_Iu(trainset, uid) for uid in df.uid]
                df['Ui'] = [get_Ui(trainset, iid) for iid in df.iid]
                df['err'] = abs(df.est - df.rui)
                best_predictions = df.sort_values(by='err')[:10]
                worst_predictions = df.sort_values(by='err')[-10:]
                
                temp_dir = tempfile.TemporaryDirectory(dir  =  outdata, prefix='predictions_')
                temp_dirname = temp_dir.name
                temp_file_best = tempfile.NamedTemporaryFile(prefix="best-predicitions_", suffix=".csv", dir=temp_dirname)
                temp_filename_best = temp_file_best.name
                temp_file_worst = tempfile.NamedTemporaryFile(prefix="worst-predicitions_", suffix=".csv", dir=temp_dirname)
                temp_filename_worst = temp_file_worst.name
                try:
                    best_predictions.to_csv(temp_filename_best, index=False)                    
                    worst_predictions.to_csv(temp_filename_worst, index=False)
                    mlflow.log_artifact(temp_dirname)
                finally:
                    temp_file_best.close()
                    temp_file_worst.close()
                    temp_dir.cleanup()
                
                    
        MlflowClient().set_tag(run_id,
                   "mlflow.note.content",
                   rundesc)

#     return run_id, experiment_id    

In [ ]:
param_grid = {'bsl_options' : {'method': ['als', 'sgd'],
              'n_epochs': [5, 15],
              'reg_u': [10, 20],
              'reg_i': [5,15]               
                }
              }

param_model = {
          'measures': ['RMSE', 'MAE'], 
          'cv':3
            }

# run_id, experiment_id = mlflow_tune_tracker(data_prep_5, 'BaselineOnly', param_grid, param_model)
mlflow_tune_tracker(data_prep_5, 'BaselineOnly', param_grid, param_model)

## Model validation

Comment the best and the worst predictions

**The best parameter combination**

Then the best model is BaselineOnly6 with 
- method: 'als'
- n_epochs: 15
- reg_u: 20
- reg_i: 5

**Compare to all the experiments we ran so we validate the model as the best one based on RMSE metric**

### Download from MLflow server the Best Model

In [ ]:
champion_name = 'BaselineOnly4'
client = MlflowClient()
experiment = client.get_experiment_by_name('Purchase Recommended System')
experiment_id = experiment.experiment_id
#Store run info in a dataframe

run = MlflowClient().search_runs(
    experiment_ids=experiment_id,
    filter_string="tags.mlflow.runName = '{}'".format(champion_name)
)

run_id = run[0].info.run_id
artifacts_list = [arts.path for arts in client.list_artifacts(run_id, path=None)]

for art_path in artifacts_list: 
    client.download_artifacts(run_id, art_path, outmodels)

### Look at Best and Worst Predictions

In [ ]:
best_predictions = pd.read_csv("/".join([outmodels, 'predictions_5qrt6nur/best-predicitions_218fjq_u.csv']))
worst_predictions = pd.read_csv("/".join([outmodels, 'predictions_5qrt6nur/worst-predicitions_kititvv8.csv']))

### Best predictions

In [ ]:
best_predictions

**Comments**: Because Ui is anywhere between 275 to 2792, it's good because we have a lot of people who buy the product (a kind of preference)

In [ ]:
fig, ax = plt.subplots()

ax.hist(data_prep_4.loc[data_prep_4['productId'] == str(best_predictions.iloc[0]['iid'])]['prod_ratings'])

ax.set_ylabel('Number of ratings')
ax.set_xlabel('Rating')
ax.set_title('Number of ratings 2 has received')
plt.show()

# # save the figure
fig.savefig(outreports + 'N_Ratings_2.png', dpi=300, bbox_inches='tight')

### Worst Predictions

In [ ]:
worst_predictions

**Comments**: Really bad. Because consider the item 148, the user buy it 0 times but the algorithm says he buy always, Also if we look at distribution only few people buy it frequently

In [ ]:
fig, ax = plt.subplots()
ax.hist(data_prep_4.loc[data_prep_4['productId'] == str(best_predictions.iloc[9]['iid'])]['prod_ratings'], color='red')
ax.set_xlabel('Rating')
ax.set_ylabel('Number of ratings')
ax.set_title('Number of ratings 211 has received')
plt.show();
# # save the figure
fig.savefig(outreports + 'N_Ratings_211.png', dpi=300, bbox_inches='tight')

### Log Figures

In [ ]:
client.log_artifact(run_id,outreports)

## Model Registration

Register the Model in the Mlflow Model Registry and set its status as STAGING

In [ ]:
model_name = 'Champion'
artifact_path = "model"
model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id, artifact_path=artifact_path)

model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

In [ ]:
client = MlflowClient()

client.update_registered_model(
  name=model_details.name,
  description="This model provides recommendation for specific users and items based on purchase data. The data consists of user transactions"
)

client.update_model_version(
  name=model_details.name,
  version=model_details.version,
  description="This model was built with Surprise library. It is a ALS based BaselineOnly algorithm"
)

client.transition_model_version_stage(
  name=model_details.name,
  version=model_details.version,
  stage='Staging',
)
model_version_details = client.get_model_version(
  name=model_details.name,
  version=model_details.version,
)
print("The current model stage is: '{stage}'".format(stage=model_version_details.current_stage))